### Parse `Tabletop Objects`

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../package/mujoco_helper/') # for 'mujoco_parser'
sys.path.append('../package/kinematics_helper/') # for 'transforms'
sys.path.append('../package/utility/') # for 'utils'
from mujoco_parser import *
from transforms import *
from utils import *
from ik import *
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:",MUJOCO_VERSION)

MuJoCo version: (3, 1, 6)


In [2]:
xml_path = '../asset/scene_objects.xml'
env = MuJoCoParserClass(name='Tabletop',rel_xml_path=xml_path,verbose=False)

### Span Camera and Capture Point Cloud Information

In [3]:
env.reset()
env.init_viewer(
    azimuth           = 163.88,
    distance          = 4.5,
    elevation         = -10,
    lookat            = [0.01,0.11,0.5],
    transparent       = False,
    black_sky         = True,
    use_rgb_overlay   = True,
    loc_rgb_overlay   = 'top right',
    use_rgb_overlay_2 = False,
    loc_rgb_overlay_2 = 'bottom right',
)

p_center = np.array([0.6, 0, 1.2])  # center point of camera rotation
radius = 0.5                        # distance from camera to rotation center point
pcd_array = np.empty((0,3))         # array to save all point cloud points (maximum number of geoms that can be rendered in MuJoCo: 10000)
num_views = 200                     # number of views to capture point cloud information (rendering can fail if too high)

R_start = rpy2r(np.radians([0,-45,0]))
obj_names = env.get_body_names(prefix='obj_')
n_obj = len(obj_names)
np.random.seed(seed=0) # FIX SEED
obj_xyzs = sample_xyzs(
    n_obj,
    x_range   = [+0.6,+1.0],
    y_range   = [-0.3,+0.3],
    z_range   = [0.802,0.802],
    min_dist  = 0.2,
    xy_margin = 0.0
)
for obj_idx in range(n_obj):
    env.set_p_base_body(body_name=obj_names[obj_idx],p=obj_xyzs[obj_idx,:])
    env.set_R_base_body(body_name=obj_names[obj_idx],R=np.eye(3,3))
env.forward(increase_tick=False)

while env.is_viewer_alive() and env.tick < 2*np.pi*num_views:
    env.step()

    theta = env.tick / num_views
    p_curr = p_center - radius* np.array((np.cos(theta),np.sin(theta),0))
    R_spin = rpy2r([0,0,theta])
    R_curr = np.dot(R_spin,R_start)
    env.data.cam('external').xpos = p_curr
    env.data.cam('external').xmat = np.reshape(R_curr,(9,))

    if env.loop_every(tick_every=10):
        rgb_ext,depth_ext,pcd_ext,T_cam_ext = env.get_fixed_cam_rgbd_pcd(
            cam_name='external',downscale_pcd=0.1)
        pcd_ext_above = pcd_ext[pcd_ext[:,2] > 0.8]
        pcd_array = np.vstack((pcd_array,pcd_ext_above))
        
    if env.loop_every(tick_every=10):
        env.plot_T(T=T_cam_ext,axis_len=0.1,axis_width=0.005,label='External')
        env.viewer.add_rgb_overlay(rgb_img_raw=rgb_ext)
        for p in pcd_ext_above: env.plot_sphere(p=p,r=0.002,rgba=[0.05,0.95,0.05,1])
        env.render()

env.close_viewer()

### Render Objects and Point Cloud Information

In [4]:
env.reset()
env.init_viewer(
    azimuth           = 163.88,
    distance          = 4.5,
    elevation         = -10,
    lookat            = [0.01,0.11,0.5],
    transparent       = False,
    black_sky         = True,
)

for obj_idx in range(n_obj):    # Objects
    env.set_p_base_body(body_name=obj_names[obj_idx],p=obj_xyzs[obj_idx,:])
    env.set_R_base_body(body_name=obj_names[obj_idx],R=np.eye(3,3))
env.forward(increase_tick=False)

while env.is_viewer_alive():
    env.step()

    env.plot_T(T=T_cam_ext,axis_len=0.1,axis_width=0.005,label='External')
    for p in pcd_array[::10]: env.plot_sphere(p=p,r=0.002,rgba=[0.05,0.95,0.05,1])  # Point Cloud
    env.render()

env.close_viewer()